# Basic Setup

In [1]:
import os
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

#import importlib.util
#spec = importlib.util.spec_from_file_location("sklearn", "/home/bhk5fs/.local/lib/python3.7/site-packages/sklearn/__init__.py")
#sklearn = importlib.util.module_from_spec(spec)
#spec.loader.exec_module(sklearn)

import sklearn

from sklearn import model_selection
from sklearn import ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

from sklearn.model_selection import cross_val_score

2021-12-21 13:19:29.008388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
print(sklearn.__version__)

0.24.1


# Load in Data

In [1]:
# Get environment variables
filenameset=str(os.getenv('filenameset'))
suffixset=str(os.getenv('suffixset'))
rootset=str(os.getenv('rootset'))
clusterfirst=str(os.getenv('clusterfirst'))
clusterlast=str(os.getenv('clusterlast'))
datagroupset=str(os.getenv('datagroupset'))

NameError: name 'os' is not defined

In [33]:
#filenameset=""
#suffixset="21-12-20"
#rootset="/project/commonappteacherrec/teacher_rec_full_replication/data/build"
#clusterfirst=str(1)
#clusterlast=str(2)
#datagroupset="train"

In [34]:
rfcpath = rootset + "/rfc.sav"
#rfcpath = "/home/bhk5fs/Common App Teacher Recommendations/rfc.sav"

In [35]:
rfc = pickle.load(open(rfcpath, "rb"))

In [36]:
filepath = rootset

In [72]:
filename1 = filepath + '/06_%s_' % datagroupset
filename1 = filename1 + 'sentiment_prep_%s.csv' % clusterfirst

In [73]:
filename2 = filepath + '/06_%s_' % datagroupset
filename2 = filename2 + 'sentiment_prep_'

In [74]:
data = pd.read_csv(filename1, sep=",", encoding='utf-8').drop(columns=["Unnamed: 0"])

In [75]:
for x in range(int(clusterfirst)+1, int(clusterlast)+1):
    filename3 = filename2 + str(x) + ".csv"
    tmp = pd.read_csv(filename3, sep=",", encoding='utf-8').drop(columns=["Unnamed: 0"])
    data = data.append(tmp)
    print(x)

2


In [76]:
data.head()

,invitationid,sentence_text,index,LABEL_0_yelp,LABEL_1_yelp,LABEL_2_yelp,LABEL_3_yelp,LABEL_4_yelp,label_predict_yelp,label_prob_yelp,...,LABEL_3_bert,LABEL_4_bert,label_predict_bert,label_prob_bert,LABEL_0_twit,LABEL_2_twit,LABEL_4_twit,label_predict_twit,label_prob_twit,label_predict_imdb
0,4479.0,"With great enthusiasm, I strongly recommend th...",0,0.003709,0.004018,0.023697,0.239652,0.728923,LABEL_4,0.728923,...,0.175286,0.811902,LABEL_4,0.811902,0.001000,0.019904,0.979097,LABEL_4,0.979097,LABEL_4
1,4479.0,While I am certain that many strong students a...,1,0.000469,0.000809,0.009739,0.164754,0.824229,LABEL_4,0.824229,...,0.574474,0.266528,LABEL_3,0.574474,0.001989,0.044409,0.953602,LABEL_4,0.953602,LABEL_4
2,4479.0,this student was a student in my English III A...,2,0.110085,0.180857,0.307634,0.219449,0.181976,LABEL_2,0.307634,...,0.429959,0.468328,LABEL_4,0.468328,0.010519,0.901323,0.088158,LABEL_2,0.901323,LABEL_4
3,4479.0,"In this challenging setting, she distinguished...",3,0.000696,0.001853,0.024479,0.227453,0.745519,LABEL_4,0.745519,...,0.343755,0.617778,LABEL_4,0.617778,0.001463,0.054443,0.944094,LABEL_4,0.944094,LABEL_4
4,4479.0,The evidence of her hard work can be seen in h...,4,0.000738,0.001292,0.010985,0.601669,0.385317,LABEL_3,0.601669,...,0.833937,0.112622,LABEL_3,0.833937,0.001434,0.050994,0.947572,LABEL_4,0.947572,LABEL_4


In [77]:
data.shape

(343624, 32)

In [78]:
data.columns

Index(['invitationid', 'sentence_text', 'index', 'LABEL_0_yelp',
       'LABEL_1_yelp', 'LABEL_2_yelp', 'LABEL_3_yelp', 'LABEL_4_yelp',
       'label_predict_yelp', 'label_prob_yelp', 'LABEL_0_xlnet',
       'LABEL_4_xlnet', 'label_predict_xlnet', 'label_prob_xlnet',
       'LABEL_0_albert', 'LABEL_4_albert', 'label_predict_albert',
       'label_prob_albert', 'label_predict_stanza', 'LABEL_0_bert',
       'LABEL_1_bert', 'LABEL_2_bert', 'LABEL_3_bert', 'LABEL_4_bert',
       'label_predict_bert', 'label_prob_bert', 'LABEL_0_twit', 'LABEL_2_twit',
       'LABEL_4_twit', 'label_predict_twit', 'label_prob_twit',
       'label_predict_imdb'],
      dtype='object')

# Clean Data for Algorithm

### Make dummy variables from predictions

In [79]:
data_x = pd.get_dummies(data, prefix=['yelp_', 'xlnet_', 'albert_', 'stanza_', 'bert_', 'twit_', 'imdb_'], columns=['label_predict_yelp', 'label_predict_xlnet', 'label_predict_albert', 'label_predict_stanza', 'label_predict_bert', 'label_predict_twit', 'label_predict_imdb'])

### Add additional predictors

In [80]:
data_x.head()

,invitationid,sentence_text,index,LABEL_0_yelp,LABEL_1_yelp,LABEL_2_yelp,LABEL_3_yelp,LABEL_4_yelp,label_prob_yelp,LABEL_0_xlnet,...,bert__LABEL_1,bert__LABEL_2,bert__LABEL_3,bert__LABEL_4,twit__LABEL_0,twit__LABEL_2,twit__LABEL_4,imdb__LABEL_0,imdb__LABEL_2,imdb__LABEL_4
0,4479.0,"With great enthusiasm, I strongly recommend th...",0,0.003709,0.004018,0.023697,0.239652,0.728923,0.728923,0.000265,...,0,0,0,1,0,0,1,0,0,1
1,4479.0,While I am certain that many strong students a...,1,0.000469,0.000809,0.009739,0.164754,0.824229,0.824229,0.000342,...,0,0,1,0,0,0,1,0,0,1
2,4479.0,this student was a student in my English III A...,2,0.110085,0.180857,0.307634,0.219449,0.181976,0.307634,0.007925,...,0,0,0,1,0,1,0,0,0,1
3,4479.0,"In this challenging setting, she distinguished...",3,0.000696,0.001853,0.024479,0.227453,0.745519,0.745519,0.000271,...,0,0,0,1,0,0,1,0,0,1
4,4479.0,The evidence of her hard work can be seen in h...,4,0.000738,0.001292,0.010985,0.601669,0.385317,0.601669,0.000300,...,0,0,1,0,0,0,1,0,0,1


In [81]:
data_x.columns

Index(['invitationid', 'sentence_text', 'index', 'LABEL_0_yelp',
       'LABEL_1_yelp', 'LABEL_2_yelp', 'LABEL_3_yelp', 'LABEL_4_yelp',
       'label_prob_yelp', 'LABEL_0_xlnet', 'LABEL_4_xlnet', 'label_prob_xlnet',
       'LABEL_0_albert', 'LABEL_4_albert', 'label_prob_albert', 'LABEL_0_bert',
       'LABEL_1_bert', 'LABEL_2_bert', 'LABEL_3_bert', 'LABEL_4_bert',
       'label_prob_bert', 'LABEL_0_twit', 'LABEL_2_twit', 'LABEL_4_twit',
       'label_prob_twit', 'yelp__LABEL_0', 'yelp__LABEL_1', 'yelp__LABEL_2',
       'yelp__LABEL_3', 'yelp__LABEL_4', 'xlnet__LABEL_0', 'xlnet__LABEL_4',
       'albert__LABEL_0', 'albert__LABEL_4', 'stanza__LABEL_0',
       'stanza__LABEL_2', 'stanza__LABEL_4', 'bert__LABEL_0', 'bert__LABEL_1',
       'bert__LABEL_2', 'bert__LABEL_3', 'bert__LABEL_4', 'twit__LABEL_0',
       'twit__LABEL_2', 'twit__LABEL_4', 'imdb__LABEL_0', 'imdb__LABEL_2',
       'imdb__LABEL_4'],
      dtype='object')

In [82]:
if 'imdb__LABEL_2' not in data_x.columns:
    data_x[['imdb__LABEL_2']] = 0
    data_x = data_x[['letterid',
 'sentence_text',
 'index',
 'LABEL_0_yelp',
 'LABEL_1_yelp',
 'LABEL_2_yelp',
 'LABEL_3_yelp',
 'LABEL_4_yelp',
 'label_prob_yelp',
 'LABEL_0_xlnet',
 'LABEL_4_xlnet',
 'label_prob_xlnet',
 'LABEL_0_albert',
 'LABEL_4_albert',
 'label_prob_albert',
 'LABEL_0_bert',
 'LABEL_1_bert',
 'LABEL_2_bert',
 'LABEL_3_bert',
 'LABEL_4_bert',
 'label_prob_bert',
 'LABEL_0_twit',
 'LABEL_2_twit',
 'LABEL_4_twit',
 'label_prob_twit',
 'yelp__LABEL_0',
 'yelp__LABEL_1',
 'yelp__LABEL_2',
 'yelp__LABEL_3',
 'yelp__LABEL_4',
 'xlnet__LABEL_0',
 'xlnet__LABEL_4',
 'albert__LABEL_0',
 'albert__LABEL_4',
 'stanza__LABEL_0',
 'stanza__LABEL_2',
 'stanza__LABEL_4',
 'bert__LABEL_0',
 'bert__LABEL_1',
 'bert__LABEL_2',
 'bert__LABEL_3',
 'bert__LABEL_4',
 'twit__LABEL_0',
 'twit__LABEL_2',
 'twit__LABEL_4',
 'imdb__LABEL_0',
 'imdb__LABEL_2',
 'imdb__LABEL_4']]

In [83]:
def add_predictors(df):
    start_time = time.time()
    
    tmp = pd.DataFrame(np.sort(df[["LABEL_0_yelp", "LABEL_1_yelp", "LABEL_2_yelp", "LABEL_3_yelp", "LABEL_4_yelp"]].values))
    df["label_prob2_yelp"] = tmp.iloc[:,-2].values
    df["label_prob_diff_yelp"] = df["label_prob_yelp"] - df["label_prob2_yelp"]

    df["label_prob_diff_xlnet"] = (df["LABEL_0_xlnet"] - df["LABEL_4_xlnet"]).abs()

    df["label_prob_diff_albert"] = (df["LABEL_0_albert"] - df["LABEL_4_albert"]).abs()

    tmp = pd.DataFrame(np.sort(df[["LABEL_0_bert", "LABEL_1_bert", "LABEL_2_bert", "LABEL_3_bert", "LABEL_4_bert"]].values))
    df["label_prob2_bert"] = tmp.iloc[:,-2].values
    df["label_prob_diff_bert"] = df["label_prob_bert"] - df["label_prob2_bert"]

    tmp = pd.DataFrame(np.sort(df[["LABEL_0_twit", "LABEL_2_twit", "LABEL_4_twit"]].values))
    df["label_prob2_twit"] = tmp.iloc[:,-2].values
    df["label_prob_diff_twit"] = df["label_prob_twit"] - df["label_prob2_twit"]
    
    print("--- %s seconds ---" % (time.time() - start_time))

    return df

In [84]:
data_x = add_predictors(data_x)

--- 0.3289480209350586 seconds ---


In [85]:
data_x.head()

,invitationid,sentence_text,index,LABEL_0_yelp,LABEL_1_yelp,LABEL_2_yelp,LABEL_3_yelp,LABEL_4_yelp,label_prob_yelp,LABEL_0_xlnet,...,imdb__LABEL_2,imdb__LABEL_4,label_prob2_yelp,label_prob_diff_yelp,label_prob_diff_xlnet,label_prob_diff_albert,label_prob2_bert,label_prob_diff_bert,label_prob2_twit,label_prob_diff_twit
0,4479.0,"With great enthusiasm, I strongly recommend th...",0,0.003709,0.004018,0.023697,0.239652,0.728923,0.728923,0.000265,...,0,1,0.239652,0.489271,0.999471,0.999219,0.175286,0.636616,0.019904,0.959193
1,4479.0,While I am certain that many strong students a...,1,0.000469,0.000809,0.009739,0.164754,0.824229,0.824229,0.000342,...,0,1,0.164754,0.659475,0.999316,0.997370,0.266528,0.307946,0.044409,0.909192
2,4479.0,this student was a student in my English III A...,2,0.110085,0.180857,0.307634,0.219449,0.181976,0.307634,0.007925,...,0,1,0.219449,0.088184,0.984150,0.983585,0.429959,0.038369,0.088158,0.813165
3,4479.0,"In this challenging setting, she distinguished...",3,0.000696,0.001853,0.024479,0.227453,0.745519,0.745519,0.000271,...,0,1,0.227453,0.518066,0.999458,0.998310,0.343755,0.274023,0.054443,0.889651
4,4479.0,The evidence of her hard work can be seen in h...,4,0.000738,0.001292,0.010985,0.601669,0.385317,0.601669,0.000300,...,0,1,0.385317,0.216352,0.999400,0.963995,0.112622,0.721315,0.050994,0.896579


In [86]:
column_check_actual = data_x.columns
column_check_actual

Index(['invitationid', 'sentence_text', 'index', 'LABEL_0_yelp',
       'LABEL_1_yelp', 'LABEL_2_yelp', 'LABEL_3_yelp', 'LABEL_4_yelp',
       'label_prob_yelp', 'LABEL_0_xlnet', 'LABEL_4_xlnet', 'label_prob_xlnet',
       'LABEL_0_albert', 'LABEL_4_albert', 'label_prob_albert', 'LABEL_0_bert',
       'LABEL_1_bert', 'LABEL_2_bert', 'LABEL_3_bert', 'LABEL_4_bert',
       'label_prob_bert', 'LABEL_0_twit', 'LABEL_2_twit', 'LABEL_4_twit',
       'label_prob_twit', 'yelp__LABEL_0', 'yelp__LABEL_1', 'yelp__LABEL_2',
       'yelp__LABEL_3', 'yelp__LABEL_4', 'xlnet__LABEL_0', 'xlnet__LABEL_4',
       'albert__LABEL_0', 'albert__LABEL_4', 'stanza__LABEL_0',
       'stanza__LABEL_2', 'stanza__LABEL_4', 'bert__LABEL_0', 'bert__LABEL_1',
       'bert__LABEL_2', 'bert__LABEL_3', 'bert__LABEL_4', 'twit__LABEL_0',
       'twit__LABEL_2', 'twit__LABEL_4', 'imdb__LABEL_0', 'imdb__LABEL_2',
       'imdb__LABEL_4', 'label_prob2_yelp', 'label_prob_diff_yelp',
       'label_prob_diff_xlnet', 'label_prob_

In [87]:
column_check_expected = ['letterid',  'sentence_text', 'index', 'LABEL_0_yelp', 'LABEL_1_yelp', 'LABEL_2_yelp', 'LABEL_3_yelp',
       'LABEL_4_yelp', 'label_prob_yelp', 'LABEL_0_xlnet', 'LABEL_4_xlnet',
       'label_prob_xlnet', 'LABEL_0_albert', 'LABEL_4_albert',
       'label_prob_albert', 'LABEL_0_bert', 'LABEL_1_bert', 'LABEL_2_bert',
       'LABEL_3_bert', 'LABEL_4_bert', 'label_prob_bert', 'LABEL_0_twit',
       'LABEL_2_twit', 'LABEL_4_twit', 'label_prob_twit', 'yelp__LABEL_0',
       'yelp__LABEL_1', 'yelp__LABEL_2', 'yelp__LABEL_3', 'yelp__LABEL_4',
       'xlnet__LABEL_0', 'xlnet__LABEL_4', 'albert__LABEL_0',
       'albert__LABEL_4', 'stanza__LABEL_0', 'stanza__LABEL_2',
       'stanza__LABEL_4', 'bert__LABEL_0', 'bert__LABEL_1', 'bert__LABEL_2',
       'bert__LABEL_3', 'bert__LABEL_4', 'twit__LABEL_0', 'twit__LABEL_2',
       'twit__LABEL_4', 'imdb__LABEL_0', 'imdb__LABEL_2', 'imdb__LABEL_4',
       'label_prob2_yelp', 'label_prob_diff_yelp', 'label_prob_diff_xlnet',
       'label_prob_diff_albert', 'label_prob2_bert', 'label_prob_diff_bert',
       'label_prob2_twit', 'label_prob_diff_twit']
column_check_expected

['invitationid',
 'sentence_text',
 'index',
 'LABEL_0_yelp',
 'LABEL_1_yelp',
 'LABEL_2_yelp',
 'LABEL_3_yelp',
 'LABEL_4_yelp',
 'label_prob_yelp',
 'LABEL_0_xlnet',
 'LABEL_4_xlnet',
 'label_prob_xlnet',
 'LABEL_0_albert',
 'LABEL_4_albert',
 'label_prob_albert',
 'LABEL_0_bert',
 'LABEL_1_bert',
 'LABEL_2_bert',
 'LABEL_3_bert',
 'LABEL_4_bert',
 'label_prob_bert',
 'LABEL_0_twit',
 'LABEL_2_twit',
 'LABEL_4_twit',
 'label_prob_twit',
 'yelp__LABEL_0',
 'yelp__LABEL_1',
 'yelp__LABEL_2',
 'yelp__LABEL_3',
 'yelp__LABEL_4',
 'xlnet__LABEL_0',
 'xlnet__LABEL_4',
 'albert__LABEL_0',
 'albert__LABEL_4',
 'stanza__LABEL_0',
 'stanza__LABEL_2',
 'stanza__LABEL_4',
 'bert__LABEL_0',
 'bert__LABEL_1',
 'bert__LABEL_2',
 'bert__LABEL_3',
 'bert__LABEL_4',
 'twit__LABEL_0',
 'twit__LABEL_2',
 'twit__LABEL_4',
 'imdb__LABEL_0',
 'imdb__LABEL_2',
 'imdb__LABEL_4',
 'label_prob2_yelp',
 'label_prob_diff_yelp',
 'label_prob_diff_xlnet',
 'label_prob_diff_albert',
 'label_prob2_bert',
 'label_pro

In [88]:
if not (np.array_equal(column_check_actual, column_check_expected)):
    raise ValueError("Columns are not as expected in the order the random forest model requires")

# Apply to teacher recommendation data

In [89]:
start_time = time.time()
rfc_predict_data = rfc.predict(data_x.drop(columns=['letterid', 'sentence_text', 'index']))
rfc_predict_data = pd.Series(rfc_predict_data)
print("--- %s seconds ---" % (time.time() - start_time))

--- 88.18238258361816 seconds ---


In [90]:
start_time = time.time()
probabilities = rfc.predict_proba(data_x.drop(columns=['letterid', 'sentence_text', 'index']))
print("--- %s seconds ---" % (time.time() - start_time))

--- 87.36851000785828 seconds ---


In [92]:
predictions = {'index': rfc_predict_data.index,'label_predict_forest': rfc_predict_data, 'LABEL_0_forest': probabilities[:,0], 'LABEL_1_forest': probabilities[:,1], 'LABEL_2_forest': probabilities[:,2], 'LABEL_3_forest': probabilities[:,3], 'LABEL_4_forest': probabilities[:,4]}

In [93]:
predictions = pd.DataFrame(predictions)

In [94]:
#data_x['merge'] = np.arange(len(data_x))

In [95]:
#data_x['merge'] = data_x.index

In [96]:
data_x.head()

,invitationid,sentence_text,index,LABEL_0_yelp,LABEL_1_yelp,LABEL_2_yelp,LABEL_3_yelp,LABEL_4_yelp,label_prob_yelp,LABEL_0_xlnet,...,imdb__LABEL_2,imdb__LABEL_4,label_prob2_yelp,label_prob_diff_yelp,label_prob_diff_xlnet,label_prob_diff_albert,label_prob2_bert,label_prob_diff_bert,label_prob2_twit,label_prob_diff_twit
0,4479.0,"With great enthusiasm, I strongly recommend th...",0,0.003709,0.004018,0.023697,0.239652,0.728923,0.728923,0.000265,...,0,1,0.239652,0.489271,0.999471,0.999219,0.175286,0.636616,0.019904,0.959193
1,4479.0,While I am certain that many strong students a...,1,0.000469,0.000809,0.009739,0.164754,0.824229,0.824229,0.000342,...,0,1,0.164754,0.659475,0.999316,0.997370,0.266528,0.307946,0.044409,0.909192
2,4479.0,this student was a student in my English III A...,2,0.110085,0.180857,0.307634,0.219449,0.181976,0.307634,0.007925,...,0,1,0.219449,0.088184,0.984150,0.983585,0.429959,0.038369,0.088158,0.813165
3,4479.0,"In this challenging setting, she distinguished...",3,0.000696,0.001853,0.024479,0.227453,0.745519,0.745519,0.000271,...,0,1,0.227453,0.518066,0.999458,0.998310,0.343755,0.274023,0.054443,0.889651
4,4479.0,The evidence of her hard work can be seen in h...,4,0.000738,0.001292,0.010985,0.601669,0.385317,0.601669,0.000300,...,0,1,0.385317,0.216352,0.999400,0.963995,0.112622,0.721315,0.050994,0.896579


In [97]:
output_prep = pd.merge(data_x, predictions, how="left", on="index", validate="1:1")

In [101]:
output_prep.head(20)

,invitationid,sentence_text,index,LABEL_0_yelp,LABEL_1_yelp,LABEL_2_yelp,LABEL_3_yelp,LABEL_4_yelp,label_prob_yelp,LABEL_0_xlnet,...,label_prob2_bert,label_prob_diff_bert,label_prob2_twit,label_prob_diff_twit,label_predict_forest,LABEL_0_forest,LABEL_1_forest,LABEL_2_forest,LABEL_3_forest,LABEL_4_forest
0,4479.0,"With great enthusiasm, I strongly recommend th...",0,0.003709,0.004018,0.023697,0.239652,0.728923,0.728923,0.000265,...,0.175286,0.636616,0.019904,0.959193,LABEL_4,0.000000,0.000000,0.001703,0.069933,0.928364
1,4479.0,While I am certain that many strong students a...,1,0.000469,0.000809,0.009739,0.164754,0.824229,0.824229,0.000342,...,0.266528,0.307946,0.044409,0.909192,LABEL_4,0.000000,0.000000,0.013982,0.300126,0.685892
2,4479.0,this student was a student in my English III A...,2,0.110085,0.180857,0.307634,0.219449,0.181976,0.307634,0.007925,...,0.429959,0.038369,0.088158,0.813165,LABEL_3,0.000000,0.000940,0.240699,0.597365,0.160995
3,4479.0,"In this challenging setting, she distinguished...",3,0.000696,0.001853,0.024479,0.227453,0.745519,0.745519,0.000271,...,0.343755,0.274023,0.054443,0.889651,LABEL_4,0.000000,0.000000,0.002482,0.085990,0.911528
4,4479.0,The evidence of her hard work can be seen in h...,4,0.000738,0.001292,0.010985,0.601669,0.385317,0.601669,0.000300,...,0.112622,0.721315,0.050994,0.896579,LABEL_4,0.000000,0.000120,0.036495,0.362092,0.601293
5,4479.0,"As is evident on her resume, this student is a...",5,0.004505,0.009757,0.088862,0.352928,0.543948,0.543948,0.000185,...,0.340183,0.260430,0.062513,0.871895,LABEL_4,0.000000,0.000000,0.007814,0.294540,0.697646
6,4479.0,While her GPA and test scores represent the re...,6,0.327366,0.254325,0.139882,0.106725,0.171702,0.327366,0.000553,...,0.443781,0.006525,0.043834,0.910450,LABEL_3,0.000000,0.000332,0.060534,0.563662,0.375472
7,4479.0,She employs the academic habits of many high-a...,7,0.000557,0.001412,0.021594,0.218321,0.758116,0.758116,0.000208,...,0.435564,0.068475,0.162095,0.669300,LABEL_4,0.000000,0.000000,0.005579,0.357720,0.636701
8,4479.0,"While her ambitious goals lay the groundwork, ...",8,0.006437,0.030480,0.200555,0.332781,0.429748,0.429748,0.003036,...,0.349733,0.146884,0.367288,0.260321,LABEL_3,0.000208,0.013794,0.390276,0.487431,0.108290
9,4479.0,this student is an incredibly talented writer ...,9,0.000821,0.001161,0.008736,0.104559,0.884723,0.884723,0.000163,...,0.153413,0.679582,0.039445,0.919075,LABEL_4,0.000000,0.000000,0.000342,0.060502,0.939156


In [100]:
outputname = rootset + "/07_%s_" % datagroupset
outputname = outputname + "forest_output_%s.csv" %suffixset

In [41]:
output_prep.to_csv(outputname)